In [28]:
import pandas as pd
import numpy as np

In [2]:
digits='0123456789'

In [3]:
def get_type_size(t):
    s=[i for i in t if i in digits]
    return int(''.join(s))

def get_simd_size(t):
    return int(t.split('x')[-1])

def get_simd_type(t):
    return t.split('x')[0]

def get_base_type(t):
    s=[i for i in t if i not in digits+'x']
    return (''.join(s))



In [4]:
def identify_retval_table(df):
    return any('Vec' in str(c) for c in df.columns) and 'Ret' in df.columns

In [5]:
def identify_parameters_table(df):
    t=df.iloc[0, 0]
    if type(t)==str:
        return t.startswith('a')
    return False

In [6]:
def is_binop(t):
    return len([i for i in t.split(',') if i.strip()])>1

In [7]:
def get_retvals_tables(url, ops, default):
    ret={}
    for op in ops:
        tables=pd.read_html(url+op+'.html')
        retval_tables=[t for t in tables if identify_retval_table(t)]
        params=[t for t in tables if identify_parameters_table(t)]
        binop=True
        if params:
            binop=is_binop(params[-1].iloc[0, 0])
        retval_table=retval_tables[-1] if retval_tables else default(binop)
        ret[op]=retval_table
    return ret

In [8]:
types=pd.DataFrame([("uint", "uint8", 16),
("int", "int8", 16),
("uint", "uint16", 8),
("int", "int16", 8),
("uint", "uint32", 4),
("int", "int32", 4),
("uint", "uint64", 2),
("int", "int64", 2),
("float", "float32", 4),
("float", "float64", 2),
("uint", "uint8", 32),
("int", "int8", 32),
("uint", "uint16", 16),
("int", "int16", 16),
("uint", "uint32", 8),
("int", "int32", 8),
("uint", "uint64", 4),
("int", "int64", 4),
("float", "float32", 8),
("float", "float64", 4),
("uint", "uint8", 64),
("int", "int8", 64),
("uint", "uint16", 32),
("int", "int16", 32),
("uint", "uint32", 16),
("int", "int32", 16),
("uint", "uint64", 8),
("int", "int64", 8),
("float", "float32", 16),
("float", "float64", 8),], columns=['base_type', 'size_type', 'size'])

In [9]:
types['width']=types.size_type.apply(get_type_size)

In [10]:
mask_types=types.iloc[:]

In [11]:
mask_types['base_type']='mask_'+mask_types['base_type']
mask_types['size_type']='mask_'+mask_types['size_type']

In [12]:
types=pd.concat([types, mask_types]).reset_index(drop=True)

In [13]:
types['floating']=types.base_type.str.contains('float')
types['mask']=types.base_type.str.contains('mask')

types['simd_type']=types.size_type+'x'+types['size'].astype(str)

types['int'] = types.base_type.str.contains('int')

In [14]:
def default_retval(ints=True, floating=True, mask=False, binop=True):
    temp = types[types.floating==floating]
    temp = temp[temp['int']==ints]
    temp = temp[types['mask']==mask]
    simd_types=temp.simd_type.drop_duplicates().values
    if binop:
        return pd.DataFrame({'Ret':simd_types, 'Vec1':simd_types, 'Vec2':simd_types})
    return pd.DataFrame({'Ret':simd_types, 'Vec':simd_types})

In [15]:
#bitwise ops
bitops_base_url='https://p12tic.github.io/libsimdpp/v2.2-dev/libsimdpp/w/bitwise/'
bitops=[
    'bit_and',
    'bit_andnot',
    'bit_or',
    'bit_xor',
    'bit_not'
]

In [16]:
bitops_retvals=get_retvals_tables(bitops_base_url, bitops, default=lambda bo: default_retval(floating=False, binop=bo))

/tmp/ipykernel_161933/456007972.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp = temp[types['mask']==mask]
/tmp/ipykernel_161933/456007972.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp = temp[types['mask']==mask]
/tmp/ipykernel_161933/456007972.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp = temp[types['mask']==mask]
/tmp/ipykernel_161933/456007972.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp = temp[types['mask']==mask]
/tmp/ipykernel_161933/456007972.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp = temp[types['mask']==mask]


In [17]:
#floating ops
floatops_base_url='https://p12tic.github.io/libsimdpp/v2.2-dev/libsimdpp/w/fp/'
floatops=['add',
 'sub',
 'mul',
 'div',
 'fmadd',
 'fmsub',
 'neg',
 'round',
 'trunc',
 'floor',
 'ceil',
 'cmp_eq',
 'cmp_neq',
 'cmp_lt',
 'cmp_gt',
 'cmp_le',
 'cmp_ge',
 'abs',
 'sign',
 'min',
 'max',
 'isnan',
 'isnan2',
 'sqrt',
 'rcp_e',
 'rcp_rh',
 'rsqrt_e',
 'rsqrt_rh',
 'reduce_add',
 'reduce_mul',
 'reduce_min',
 'reduce_max']

In [18]:
floatops_retvals=get_retvals_tables(floatops_base_url, floatops, default=lambda bo: default_retval(False, binop=bo))

In [19]:
#int ops
intops_base_url='https://p12tic.github.io/libsimdpp/v2.2-dev/libsimdpp/w/int/'
intops=['add',
 'sub',
 'add_sat',
 'sub_sat',
 'mul_lo',
 'mul_hi',
 'mull',
 'neg',
 'cmp_eq',
 'cmp_neq',
 'cmp_lt',
 'cmp_gt',
 'cmp_le',
 'cmp_ge',
 'abs',
 'avg',
 'avg_trunc',
 'min',
 'max',
 'shift_l',
 'shift_r',
 'reduce_add',
 'reduce_mul',
 'reduce_min',
 'reduce_max',
 'reduce_and',
 'reduce_or']

In [20]:
intops_retvals=get_retvals_tables(intops_base_url, intops, default=lambda bo: default_retval(True, False, binop=bo))

In [21]:
ns="simdpp::"

In [22]:
def get_best_type_table(t):
    a = types[types.size_type==t]
    b = types[types.simd_type==t]
    if len(a)>len(b):
        return a
    return b

def get_type_combinations(ts):
#     import pdb; pdb.set_trace()
    ts=[get_best_type_table(t) for t in ts]
    sizes=set(ts[0]['size'].values)
    for t in ts[1:]:
        sizes = sizes.intersection(set(t['size'].values))
    ts=[t[t['size'].isin(sizes)] for t in ts]
    for s in sizes:
        ret=[]
        for t in ts:
            ret.append(t[t['size']==s].iloc[0]['simd_type'])
        yield ret

In [23]:
def type_lookup(kt, k, vt):
    return types[types[kt]==k][vt].iloc[0]

In [73]:
aliases={
    'add':'`+`',
    'sub':'`-`',
    'neg':'`-`',
    'mul':'`*`',
    'div':'`/`',
    'divide':'`/`',
    'mull':'`*`',
    'shift_l':'`<<`',
    'shift_r':'`>>`',
     'cmp_eq': '`==`',
     'cmp_neq': '`~=`',
     'cmp_lt': '`<`',
     'cmp_gt': '`>`',
     'cmp_le': '`<=`',
     'cmp_ge': '`>=`',
    'bit_and': '`&`',
    'bit_andnot': '`&~`',
    'bit_or': '`|`',
    'bit_xor': '`^`',
    'bit_not': '`~`',
    'fmadd': '`*+`',
    'fmsub': '`*-`',
}

special={
    'fmadd': 'a * b + c',
    'fmsub': 'a * b - c',
}

In [122]:
def get_numpy_res(op, a, b = None, c = None):
    if op in aliases:
        op=aliases[op]
    if op in special:
        op = special[op]
        return eval(op)
    op=op.replace('`', '')
    if b is not None:
        return eval(f"a {op} b")
    try:
        return eval(f"np.{op}(a)")
    except:
        try:
            return eval(f"{op} a")
        except:
            return eval(f"a.{op}()")

In [123]:
def gen_bin_op(op, op_name, arg_type, ret_type, arg2_type=None):
    if arg2_type is None: arg2_type=arg_type
    size1 = type_lookup('simd_type', arg_type, 'size')
    size2 = type_lookup('simd_type', arg2_type, 'size')
    size_type1 = type_lookup('simd_type', arg_type, 'size_type')
    size_type2 = type_lookup('simd_type', arg2_type, 'size_type')
    arg1 = np.random.randint(-100, 100, size=size1).astype(size_type1)
    arg2 = np.random.randint(-100, 100, size=size2).astype(size_type2)
    res = get_numpy_res(op, arg1, arg2).tolist()
    arg1=[str(i) for i in arg1]
    arg2=[str(i) for i in arg2]
    simd_arg1 = arg1[:]
    simd_arg1[0] = f'{size_type1}({simd_arg1[0]})'
    simd_arg1 = f"Vector({', '.join(simd_arg1)})"
    simd_arg2 = arg2[:]
    simd_arg2[0] = f'{size_type2}({simd_arg2[0]})'
    simd_arg2 = f"Vector({', '.join(simd_arg2)})"
    return f'''
test "op: {op}, op_name: {op_name}, arg_type: {arg_type}, ret_type: {ret_type}, arg2_type: {arg2_type}":
  check ({op}({simd_arg1}, {simd_arg2})).to_array == {res}'''

def gen_tri_op(op, op_name, arg_type, ret_type, arg2_type=None, arg3_type=None):
    if arg2_type is None: arg2_type=arg_type
    if arg3_type is None: arg3_type=arg2_type
    size1 = type_lookup('simd_type', arg_type, 'size')
    size2 = type_lookup('simd_type', arg2_type, 'size')
    size3 = type_lookup('simd_type', arg3_type, 'size')
    size_type1 = type_lookup('simd_type', arg_type, 'size_type')
    size_type2 = type_lookup('simd_type', arg2_type, 'size_type')
    size_type3 = type_lookup('simd_type', arg3_type, 'size_type')
    arg1 = np.random.randint(-100, 100, size=size1).astype(size_type1)
    arg2 = np.random.randint(-100, 100, size=size2).astype(size_type2)
    arg3 = np.random.randint(-100, 100, size=size3).astype(size_type3)
    res = get_numpy_res(op, arg1, arg2, arg3).tolist()
    arg1=[str(i) for i in arg1]
    arg2=[str(i) for i in arg2]
    arg3=[str(i) for i in arg3]
    simd_arg1 = arg1[:]
    simd_arg1[0] = f'{size_type1}({simd_arg1[0]})'
    simd_arg1 = f"Vector({', '.join(simd_arg1)})"
    simd_arg2 = arg2[:]
    simd_arg2[0] = f'{size_type2}({simd_arg2[0]})'
    simd_arg2 = f"Vector({', '.join(simd_arg2)})"
    simd_arg3 = arg3[:]
    simd_arg3[0] = f'{size_type3}({simd_arg3[0]})'
    simd_arg3 = f"Vector({', '.join(simd_arg3)})"
    return f'''
test "op: {op}, op_name: {op_name}, arg_type: {arg_type}, ret_type: {ret_type}, arg2_type: {arg2_type}, arg3_type: {arg3_type}":
  check ({op}({simd_arg1}, {simd_arg2}, {simd_arg3})).to_array == {res}'''

def gen_uni_op(op, op_name, arg_type, ret_type):
    size1 = type_lookup('simd_type', arg_type, 'size')
    size_type1 = type_lookup('simd_type', arg_type, 'size_type')
    arg1 = np.random.randint(-100, 100, size=size1).astype(size_type1)
    res = get_numpy_res(op, arg1).tolist()
    arg1=[str(i) for i in arg1]
    simd_arg1 = arg1[:]
    simd_arg1[0] = f'{size_type1}({simd_arg1[0]})'
    simd_arg1 = f"Vector({', '.join(simd_arg1)})"
    return f'''
test "op: {op}, op_name: {op_name}, arg_type: {arg_type}, ret_type: {ret_type}":
  check ({op}({simd_arg1})).to_array == {res}'''

In [124]:
print(gen_tri_op('fmadd', 'fmadd', 'float32x4', 'float32x4', 'float32x4'))


test "op: fmadd, op_name: fmadd, arg_type: float32x4, ret_type: float32x4, arg2_type: float32x4, arg3_type: float32x4":
  check (fmadd(Vector(float32(-97.0), 50.0, -54.0, -6.0), Vector(float32(34.0), 64.0, 57.0, 35.0), Vector(float32(69.0), 64.0, -46.0, -68.0))).to_array == [-3298.0, 3200.0, -3078.0, -210.0]


In [125]:
print(gen_bin_op('`*`', 'add', 'float32x4', 'float32x4'))


test "op: `*`, op_name: add, arg_type: float32x4, ret_type: float32x4, arg2_type: float32x4":
  check (`*`(Vector(float32(-93.0), 17.0, 15.0, -74.0), Vector(float32(-5.0), -97.0, -81.0, 71.0))).to_array == [465.0, -1649.0, -1215.0, -5254.0]


In [126]:
print(gen_uni_op('`~`', 'bit_not', 'uint32x4', 'uint32x4'))


test "op: `~`, op_name: bit_not, arg_type: uint32x4, ret_type: uint32x4":
  check (`~`(Vector(uint32(31), 58, 99, 60))).to_array == [4294967264, 4294967237, 4294967196, 4294967235]


In [127]:
keywords={'div':'divide'}

In [128]:
def name_op(op):
    if op in keywords:
        return keywords[op]
    return op

In [129]:
def gen_op(op, retvals):
#     import pdb; pdb.set_trace()
    binop=retvals.shape[1]>2 and 'Vec3' not in retvals.columns
    triop=retvals.shape[1]>3 and 'Vec3' in retvals.columns
    if binop:
        for i in range(len(retvals)):
            temp=retvals.iloc[i]
            for c in get_type_combinations([temp['Ret'], temp['Vec1'], temp['Vec2']]):
                output(gen_bin_op(name_op(op), op, c[1], c[0], c[2]))
                if op in aliases:
                    output(gen_bin_op(aliases[op], op, c[1], c[0], c[2]))
    elif triop:
        for i in range(len(retvals)):
            temp=retvals.iloc[i]
            for c in get_type_combinations([temp['Ret'], temp['Vec1'], temp['Vec2'], temp['Vec3']]):
                output(gen_tri_op(name_op(op), op, c[1], c[0], c[2], c[3]))
                if op in aliases:
                    output(gen_tri_op(aliases[op], op, c[1], c[0], c[2], c[3]))
    else:
        for i in range(len(retvals)):
            temp=retvals.iloc[i].to_dict()
            vec=[v for k,v in temp.items() if 'Vec' in k][0]
            for c in get_type_combinations([temp['Ret'], vec]):
                output(gen_uni_op(name_op(op), op, c[1], c[0]))
                if op in aliases:
                    output(gen_uni_op(aliases[op], op, c[1], c[0]))

In [130]:
size_types=types[types['mask']==False].size_type.unique()
simd_types=types[types['mask']==False].simd_type.unique()

In [131]:
def gen_conversions():
    ret=[]
    for t1 in simd_types:
        for t2 in simd_types:
            size1=get_simd_size(t1)
            size2=get_simd_size(t2)
            t=get_simd_type(t2)
            if size1==size2:
                if t1!=t2:
                    output(
                        f'proc to_{t}*(x: {t1}): {t2} {{.header: simd, importcpp: "@.to_{t}()".}}'
                    )

In [132]:
def gen_simd_types(s):
#     import pdb; pdb.set_trace()
    if not s['mask']:
        gen_simd_type(s['base_type'], s['size_type'], s['size'])

In [133]:
all_output=""
def output(s):
    global all_output
    all_output+=s+'\n'

In [86]:
%pdb

Automatic pdb calling has been turned OFF


In [138]:
for retvals in [bitops_retvals, floatops_retvals, intops_retvals]:
    for k, v in retvals.items():
        try:
            gen_op(k, v)
        except (TypeError, SyntaxError, AttributeError):
            pass

<string>:1: RuntimeWarning: invalid value encountered in sqrt


In [140]:
def dedup(ls):
    curr=[]
    for l in ls:
        if l not in curr:
            curr.append(l)
        elif "###" in l:
            curr.append(l)
    return curr


with open('./tests/tests.nim', 'a+') as f:
    f.write('''
# To run these tests, simply execute `nimble test`.

import unittest

import nimd
''')
    for l in dedup([i for i in all_output.split('\n') if i.strip()]):
        f.write(l)
        f.write('\n')

In [47]:
types[['base_type', 'size_type', 'size', 'simd_type']].sort_values(['base_type', 'size'])#.to_html().replace('\n', '')

,base_type,size_type,size,simd_type
9,float,float64,2,float64x2
8,float,float32,4,float32x4
19,float,float64,4,float64x4
18,float,float32,8,float32x8
29,float,float64,8,float64x8
28,float,float32,16,float32x16
7,int,int64,2,int64x2
5,int,int32,4,int32x4
17,int,int64,4,int64x4
3,int,int16,8,int16x8


In [23]:
import numpy as np

In [25]:
(np.array([1,2,3], dtype=np.float32)>>1).tolist()

TypeError: ufunc 'right_shift' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [11]:
np.multiply?


In [17]:
3>>1

1